In [25]:
#!pip install pandas plotly scikit-learn optuna ipywidgets ipykernel nbformat matplotlib pingouin gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
#EDA
import pandas as pd
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)

#Machine Learning
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

#Otimização HP
import optuna

### Carregar os dados

In [ ]:
#Carregamento dos dados (dataset)
df_clientes = pd.read_csv('dataset.csv')

In [4]:
#Visualizar a estrutura
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [5]:
#Visualizar os primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


### EDA

In [6]:
#Distribuição da variável inovação
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [7]:
#Teste ANOVA (Análise de Variância)
#Verificar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
#Suposição / Pressuposto
# - Observações independentes
# - Variável dependente é contínua
# - Segue uma distribuição normal
# - Homogeneidade das variâncias
# - Amostras sejam de tamanhos iguais

In [8]:
#Checar se as varáncias (faturamento) entre os grupos (inovação) são homogêneas
#Aplicar Teste de Bartlett
#H0 - Variâncias são iguais
#H1 - Variâncias não são iguais

from scipy.stats import bartlett

#Separando os dados de faturamento em grupos com base na coluna inovação
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] == grupo] for grupo in df_clientes['inovacao'].unique()]

#Executar o teste de Bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*dados_agrupados)

#Exibição dos resultados
print(f'Estatistica do Teste de Bartlett: {bartlett_test_statistic}')
print(f'P-Value do Teste de Bartlett: {bartlett_p_value}')

Estatistica do Teste de Bartlett: 10.901203117231173
P-Value do Teste de Bartlett: 0.28254182954905804


In [9]:
#Executar o Teste de Shapiro-Wilk
#Verificar se os dados seguem uma distribuição normal
#H0 - Segue uma distribuição normal
#H1 - Não segue uma distribuição normal

from scipy.stats import shapiro

#Executar o teste Shapiro-Wilk
shapiro_test_statistic, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

#Exibição dos resultados
print(f'Estatistica do Teste de SW: {shapiro_test_statistic}')
print(f'P-Value do Teste de SW: {shapiro_p_value}')

Estatistica do Teste de SW: 0.9959857602472711
P-Value do Teste de SW: 0.23513451034389005


In [10]:
#Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
#H0 - Não há diferenças significativas entre as médias dos grupos
#H1 - Há pelo menos uma diferença significativa entre as médias dos grupos
from pingouin import welch_anova

aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clientes)

#Exibindo os resultados
print(f'Estatistica do Teste de ANOVA Welch: {aov.loc[0, "F"]}')
print(f'P-Value do Teste de ANOVA Welch: {aov.loc[0, "p-unc"]}')

Estatistica do Teste de ANOVA Welch: 1.1269836194061693
P-Value do Teste de ANOVA Welch: 0.3452621127391258


### Treinamendo do algoritmo K-Means

In [11]:
#Selecionar as colunas para clusterização
X = df_clientes.copy()

#Separando as variáveis numéricas, categoricas e ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

#Aplicar Transformações por tipo
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

#Transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [12]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])

In [13]:
#Optuna para otimização de hiperparâmetros
def kmeans_objective(trial):
  n_clusters = trial.suggest_int('n_clusters', 3, 10)
  distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])

  #Criar modelo
  modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)

  #Treinamendo do modelo
  modelo_kmeans.fit(X_transformed)

  #Calculando o Silhouette Score
  distances = pairwise_distances(X_transformed, metric=distance_metric)
  silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_)

  return silhouette_avg

In [14]:
#Criação de um estudo do Optuna
search_space = {'n_clusters': [3, 4, 5, 6, 7, 8, 9, 10], 'distance_metric': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

#Execução do estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-02-05 01:54:58,620] A new study created in memory with name: no-name-fb3f759a-ad1b-4d3b-be21-ebd7866879bc
[I 2025-02-05 01:54:58,899] Trial 0 finished with value: 0.38478794965029234 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.38478794965029234.
[I 2025-02-05 01:54:58,965] Trial 1 finished with value: 0.11230048381589083 and parameters: {'n_clusters': 9, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.38478794965029234.
[I 2025-02-05 01:54:59,093] Trial 2 finished with value: 0.4445458290999088 and parameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2025-02-05 01:54:59,180] Trial 3 finished with value: 0.38478794965029234 and parameters: {'n_clusters': 4, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4445458290999088.
[I 2025-02-05 01:54:59,270] Trial 4 finished with value: 0.14731572416665722 and parameters: {'n_clusters': 8, 'dista

In [16]:
#Melhor configuração encontrada pelo Optuna
best_params = estudo_kmeans.best_params

#Instanciando o modelo K-Means com melhores parâmetros
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_transformed)

#Calculando o Silhouette Score
distances = pairwise_distances(X_transformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f'k (Numero de Cluesters): {best_params["n_clusters"]}')
print(f'k (Numero de Cluesters): {best_params["distance_metric"]}')
print(f'Silhouette Score: {best_silhouette}')

k (Numero de Cluesters): 3
k (Numero de Cluesters): minkowski
Silhouette Score: 0.4445458290999088


In [18]:
#Criar coluna com cluster escolhido
df_clientes['cluster'] = best_kmeans.labels_

In [19]:
#Visualizar os primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar Resultados

In [20]:
#Cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster')

In [21]:
#Cruzar inovação e faturamento, apresentando os clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster')

In [22]:
#Cruzar número de funcionários e faturamento, apresentando os clusters
px.scatter(df_clientes, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster')

### Salvar o Modelo e o Pipeline de Transfromação

In [23]:
import joblib

#Salvar o modelo
joblib.dump(best_kmeans, 'modelo_clusterizacao_clientes.pkl')

#Salvar o Pipeline
joblib.dump(preprocessor, 'pipeline_clusterizacao_clientes.pkl')

['pipeline_clusterizacao_clientes.pkl']

### Aplicação Batch no Gradio

In [26]:
import gradio as gr
modelo = joblib.load('./modelo_clusterizacao_clientes.pkl')
preprocessor = joblib.load('./pipeline_clusterizacao_clientes.pkl')

def clustering(arquivo):
  #carregamento do csv em um dataframe
  df_empresas = pd.read.csv(arquivo.name)

  #transformação dos dados do dataframe para o formato que o KMeans precisa
  X_tranformed = preprocessor.fit_transform(df_empresas)
  modelo.fit(X_transformed)
  df_empresas['cluster'] = modelo.labels_
  df_empresas.to_csv('./clusters.csv', index=False)
  return './clusters.csv'


In [28]:
#Criação da interface
app = gr.Interface(
    clustering,
    gr.File(file_types=[".csv"]),
    "file"
)

app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9de17a33a011709297.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


KeyboardInterrupt: 